In [1]:
import pandas as pd
import numpy as np

In [2]:
X= pd.read_pickle('df2')
X

,crs_dep_timeFT,crs_arr_timeFT,crs_elapsed_timeFT,distanceFT,origin,dest,carrierAvgDelay,MonthAvgDelay
0,0.347826,0.375000,0.039113,0.070300,2.066359,2.973759,3.549976,8.977515
1,0.391304,0.625000,0.110169,0.233017,2.019957,4.324298,7.155529,4.159131
2,0.695652,0.708333,0.039765,0.042654,5.520500,7.378240,7.155529,2.993607
3,0.565217,0.625000,0.054759,0.076619,4.436164,11.047437,6.192538,2.853585
4,0.739130,0.791667,0.072360,0.118088,5.245363,6.338766,11.294149,6.511144
...,...,...,...,...,...,...,...,...
89072,0.652174,0.666667,0.039113,0.064179,2.779153,6.359470,3.549976,1.708452
89073,0.913043,0.000000,0.066493,0.135861,10.971327,18.860065,7.031039,2.853585
89074,0.782609,0.875000,0.048892,0.100908,5.258827,6.667580,9.682003,3.958788
89075,0.739130,0.791667,0.041069,0.038507,7.065939,1.995582,7.457289,10.414444


In [3]:
#X= pd.read_csv('../Data Prep Files/X.csv', index_col=0)
ydf= pd.read_csv('../Data Prep Files/y_test100000noDum.csv', compression='gzip')
ydf

,yFT,y_sign
0,0.318681,NaN
1,0.527473,0.0
2,0.340659,0.0
3,0.373626,0.0
4,0.439560,0.0
...,...,...
89072,0.472527,0.0
89073,0.439560,1.0
89074,0.340659,1.0
89075,0.406593,0.0


In [4]:
y=ydf['yFT']
#y=ydf['y_sign']
y

0        0.318681
1        0.527473
2        0.340659
3        0.373626
4        0.439560
           ...   
89072    0.472527
89073    0.439560
89074    0.340659
89075    0.406593
89076    0.527473
Name: yFT, Length: 89077, dtype: float64

In [5]:
#y.describe()

In [6]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1)

In [7]:
from xgboost import XGBRegressor
xgboost = XGBRegressor(n_estimators=100,learning_rate=0.1,reg_alpha=8)
xgboost.fit(X_train, y_train)
y_pred= xgboost.predict(X_test)

In [8]:
from sklearn.metrics import r2_score, mean_squared_error

print('Training R-Square', xgboost.score(X_train, y_train))
print('RMSE: ', mean_squared_error(y_test, y_pred))
print('R-squared score: ', r2_score(y_test, y_pred))

Training R-Square 0.06967824844700765
RMSE:  0.02712483615045412
R-squared score:  0.041358373357538


In [23]:
parameters = {
    'n_estimators':[50,100,500], 
    'learning_rate':[0.01,0.1,1],
    #'reg_alpha' :[1,8,20],
    'reg_lambda' :[1,8,20]
}

In [24]:
from sklearn.model_selection import GridSearchCV
clf = GridSearchCV(XGBRegressor(), parameters,verbose=3,n_jobs=-2) #if running overnight, use n_jobs=-1 for max speed.
clf.fit(X,y)

Fitting 5 folds for each of 27 candidates, totalling 135 fits


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=-2)]: Done  26 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-2)]: Done 122 tasks      | elapsed: 15.5min
[Parallel(n_jobs=-2)]: Done 135 out of 135 | elapsed: 19.0min finished


GridSearchCV(estimator=XGBRegressor(base_score=None, booster=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, gamma=None,
                                    gpu_id=None, importance_type='gain',
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=None, min_child_weight=None,
                                    missing=nan, monotone_constraints=None,
                                    n_estimators=100, n_jobs=None,
                                    num_parallel_tree=None, random_state=None,
                                    reg_alpha=None, reg_lambda=None,
                                    scale_pos_weight=None, subsample=None,
                                    tree_method=None, validate_parameters=None,
      

In [26]:
import pickle

In [33]:
pickle.dump(clf,open('GridSearchPickle', 'wb'))

In [35]:
testCLF= pickle.load(open('GridSearchPickle', 'rb'))
testCLF

GridSearchCV(estimator=XGBRegressor(base_score=None, booster=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, gamma=None,
                                    gpu_id=None, importance_type='gain',
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=None, min_child_weight=None,
                                    missing=nan, monotone_constraints=None,
                                    n_estimators=100, n_jobs=None,
                                    num_parallel_tree=None, random_state=None,
                                    reg_alpha=None, reg_lambda=None,
                                    scale_pos_weight=None, subsample=None,
                                    tree_method=None, validate_parameters=None,
      

In [37]:
clf.best_params_

{'learning_rate': 0.1, 'n_estimators': 100, 'reg_lambda': 8}

In [19]:
#X.isnull().sum()

(crs_dep_timeFT        0
 crs_arr_timeFT        0
 crs_elapsed_timeFT    0
 distanceFT            0
 origin                0
 dest                  0
 carrierAvgDelay       0
 MonthAvgDelay         0
 dtype: int64,
 0)

In [20]:
#y.isnull().sum()

0